In [14]:
import pandas as pd
import numpy as np
from pybrain.tools.shortcuts import buildNetwork
from pybrain.datasets import SupervisedDataSet
from pybrain.supervised.trainers import BackpropTrainer
import quadratic_weighted_kappa
import feature_generator
from pybrain.structure import FeedForwardNetwork
from pybrain.structure import LinearLayer, SigmoidLayer
from pybrain.structure import FullConnection
from pybrain.structure import BiasUnit

In [2]:
dfTrain = pd.read_csv('train.csv')
dfTest = pd.read_csv('test.csv')    

In [3]:
features = feature_generator.GetFeatures(dfTrain, dfTest)

Product_Info_2
Product_Info_3
Employment_Info_2
InsuredInfo_3
Medical_History_2
Medical_History_10


In [51]:
ds = SupervisedDataSet(len(features), 8)

In [36]:
ds['input'], ds['target']

(array([], shape=(0, 227), dtype=float64),
 array([], shape=(0, 1), dtype=float64))

In [55]:
# target = dfTrain['Response'].values
target = np.zeros((len(dfTrain), 8))
for i in range(len(dfTrain)):
        target[i,dfTrain.iloc[i]['Response'] - 1] = 1
        target[i] = dfTrain.iloc[i]['Response']

In [56]:
for i in range(len(dfTrain)):
    ds.addSample(dfTrain.iloc[i][features].values, target[i])

In [57]:
net = buildNetwork(len(features),64,1, bias=True)

# net = FeedForwardNetwork()
# inLayer = LinearLayer(len(features))
# hiddenLayer1 = SigmoidLayer(128)
# hiddenLayer2 = SigmoidLayer(64)
# outLayer = LinearLayer(8)

# net.addInputModule(inLayer)
# net.addModule(hiddenLayer1)
# net.addModule(hiddenLayer2)
# net.addOutputModule(outLayer)
# net.addModule(BiasUnit(name = 'bias'))

# in_to_hidden = FullConnection(inLayer, hiddenLayer1)
# hidden1_to_hidden2 = FullConnection(hiddenLayer1, hiddenLayer2)
# hidden_to_out = FullConnection(hiddenLayer2, outLayer)

# net.addConnection(in_to_hidden)
# # net.addConnection(FullConnection(net['bias'], hiddenLayer1))
# # net.addConnection(FullConnection(net['bias'], hiddenLayer2))
# net.addConnection(hidden1_to_hidden2)
# net.addConnection(hidden_to_out)

# net.sortModules()

trainer = BackpropTrainer(net, ds, verbose=True)

AssertionError: 

In [ ]:
error = trainer.train()
errors = [error,]
trainer.trainUntilConvergence(maxEpochs = 100)
# for iterations in range(3):
#     print iterations, errors[-1]
#     errors.append(trainer.train())

In [ ]:
predictions = trainer.testOnClassData()

In [ ]:
predictions = np.asarray(predictions) + 1
# predictions = np.clip(np.asarray(predictions), 1, 8)
print predictions

In [44]:
print quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, dfTrain['Response'])

0.0


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 